In [1]:
# Add ticket score to existing SQLite tables
import sqlite3
import pandas as pd

In [2]:
# Load the scored tickets CSV
csv_path = "scored_tickets.csv"
df_scores = pd.read_csv(csv_path)

# Normalize column names just in case
df_scores.columns = [col.strip().lower().replace(" ", "_") for col in df_scores.columns]

# Define the tables to update
tables = ["tickets", "ticketsbsb", "ticketscon", "ticketsfb"]

# Connect to the database
conn = sqlite3.connect("tickets.db")
cursor = conn.cursor()

In [3]:
for table in tables:
    print(f"Processing table: {table}")

    # Add the column if it doesn't exist
    cursor.execute(f"PRAGMA table_info({table})")
    cols = [row[1] for row in cursor.fetchall()]
    if "Ticket Score" not in cols:
        cursor.execute(f"ALTER TABLE {table} ADD COLUMN 'Ticket Score' TEXT")
        conn.commit()

    # Load table data
    df_table = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df_table.columns = [col.strip().lower().replace(" ", "_") for col in df_table.columns]

    updates = 0
    for index, row in df_table.iterrows():
        match = df_scores[(df_scores["id"] == row["id"]) &
                          (df_scores["section"] == row["section"]) &
                          (df_scores["row"] == row["row"]) &
                          (df_scores["price"] == row["price"]) &
                          (df_scores["date"] == row["date"]) &
                          (df_scores["home_team"] == row["home_team"]) &
                          (df_scores["away_team"] == row["away_team"])]

        if not match.empty:
            score = match.iloc[0]["predicted_score_10"]
        else:
            score = "no score"

        # Update the record in the database
        cursor.execute(f"""
            UPDATE {table}
            SET "Ticket Score" = ?
            WHERE id = ?
        """, (score, row["id"]))
        updates += 1

    print(f"Updated {updates} rows in {table}.")
    conn.commit()

cursor.close()
conn.close()

Processing table: tickets
Updated 13939 rows in tickets.
Processing table: ticketsbsb
Updated 75561 rows in ticketsbsb.
Processing table: ticketscon
Updated 7929 rows in ticketscon.
Processing table: ticketsfb
Updated 2032 rows in ticketsfb.


In [4]:
print("All ticket tables updated with 'Ticket Score'.")

All ticket tables updated with 'Ticket Score'.
